In [4]:
from dotenv import load_dotenv
import os
import getpass

load_dotenv()

True

In [5]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [6]:
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGSMITH_TRACING"] = "true"

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    max_retries=2,
)

In [8]:
result=llm.invoke('What is two sum problem? Explain in 50 word or less.')

In [9]:
result.content

"The Two Sum problem asks you to find two numbers in a given array of integers that add up to a specific target sum. Your goal is to return the **indices** of these two numbers. It's a fundamental problem often used to test basic algorithmic thinking."

The Two Sum problem asks you to find two numbers in a given array of integers that sum up to a specific target number. You typically return the indices of these two numbers. It's a classic problem, often solved efficiently using a hash map.

In [10]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

In [11]:
python_repl = PythonREPL()

In [12]:
python_repl.run("print(1+1)")

Python REPL can execute arbitrary code. Use with caution.


'2\n'

In [13]:
python_repl.run("""
def fib(n):
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2)

fib(10)
""")


'NameError("name \'fib\' is not defined")'

In [60]:
from langchain_experimental.utilities import PythonREPL
import sys
from io import StringIO

class PersistentPythonREPLTool(PythonREPL):
    def __init__(self):
        super().__init__()
        self._globals = {}

    def run(self, command: str) -> str:
        # Capture stdout
        old_stdout = sys.stdout
        sys.stdout = captured_output = StringIO()
        
        try:
            # First try eval (single expression)
            result = eval(command, self._globals)
            output = captured_output.getvalue()
            if output:
                return output + str(result)
            return str(result)
        except SyntaxError:
            # If it's a statement block, use exec
            try:
                exec(command, self._globals)
                output = captured_output.getvalue()
                return output if output else "Executed successfully with no output."
            except Exception as e:
                output = captured_output.getvalue()
                return output + repr(e) if output else repr(e)
        except Exception as e:
            output = captured_output.getvalue()
            return output + repr(e) if output else repr(e)
        finally:
            # Restore stdout
            sys.stdout = old_stdout

In [61]:
# Create a new instance with the fixed version
python_repl = PersistentPythonREPLTool()

# Test the fixed tool with print statements
print("Testing fixed REPL tool:")
print(python_repl.run("""
def twoSum(nums, target):
    print("Running Two Sum function...")
    num_map = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in num_map:
            return [num_map[complement], i]
        num_map[num] = i
    return []

print("Function defined successfully!")
result = twoSum([2, 7, 11, 15], 9)
print(f"Result: {result}")
"""))


Testing fixed REPL tool:
Function defined successfully!
Running Two Sum function...
Result: [0, 1]



In [62]:
# You can create the tool to pass to an agent with the fixed python_repl
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be" \
                " a valid python command. If you want to see the output of a value, you " \
                "should print it out with `print(...)`.",
    func=python_repl.run,
)

In [63]:
llm_with_tools=llm.bind_tools([repl_tool])

In [64]:
result=llm_with_tools.invoke("What is two sum problem? Explain in 50 word or less. and run the code with the tool to show me some test cases")

In [65]:
result

AIMessage(content='The Two Sum problem asks you to find two numbers in an array that add up to a specific target. You need to return the indices of these two numbers. Each input has exactly one solution, and you cannot use the same element twice.\n\nHere are some test cases:', additional_kwargs={'function_call': {'name': 'python_repl', 'arguments': '{"__arg1": "\\ndef two_sum(nums, target):\\n    num_map = {}\\n    for i, num in enumerate(nums):\\n        complement = target - num\\n        if complement in num_map:\\n            return [num_map[complement], i]\\n        num_map[num] = i\\n\\nprint(f\\"Test Case 1: nums=[2, 7, 11, 15], target=9 -> {two_sum([2, 7, 11, 15], 9)}\\")\\nprint(f\\"Test Case 2: nums=[3, 2, 4], target=6 -> {two_sum([3, 2, 4], 6)}\\")\\nprint(f\\"Test Case 3: nums=[3, 3], target=6 -> {two_sum([3, 3], 6)}\\")\\n"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safet

AIMessage(content='The Two Sum problem asks you to find two numbers in an array that add up to a specific target. You need to return the indices of these two numbers.
\n\nHere are some test cases:',

 additional_kwargs={'function_call': {'name': 'python_repl', 'arguments': '{"__arg1": 
 
 "\\ndef two_sum(nums, target):\\n    num_map = {}\\n    for i, num in enumerate(nums):\\n        complement = target - num\\n        if complement in num_map:\\n            return [num_map[complement], i]\\n        num_map[num] = i\\n    return []\\n\\nprint(two_sum([2, 7, 11, 15], 9))\\nprint(two_sum([3, 2, 4], 6))\\nprint(two_sum([3, 3], 6))\\nprint(two_sum([1, 2, 3, 4, 5], 10))\\n"}'}},
 
 
  response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--eb69a9d0-4c52-4f58-916f-92ed58ba8b82-0',
  
   tool_calls=[{'name': 'python_repl', 'args': {'__arg1': '\ndef two_sum(nums, target):\n    num_map = {}\n    for i, num in enumerate(nums):\n        complement = target - num\n        if complement in num_map:\n            return [num_map[complement], i]\n        num_map[num] = i\n    return []\n\nprint(two_sum([2, 7, 11, 15], 9))\nprint(two_sum([3, 2, 4], 6))\nprint(two_sum([3, 3], 6))\nprint(two_sum([1, 2, 3, 4, 5], 10))\n'}, 
   
   
   'id': 'fca36480-fd74-4c35-a973-8f0ecc450a87', 'type': 'tool_call'}], usage_metadata={'input_tokens': 105, 'output_tokens': 311, 'total_tokens': 416, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 100}})

In [66]:
result.tool_calls

[{'name': 'python_repl',
  'args': {'__arg1': '\ndef two_sum(nums, target):\n    num_map = {}\n    for i, num in enumerate(nums):\n        complement = target - num\n        if complement in num_map:\n            return [num_map[complement], i]\n        num_map[num] = i\n\nprint(f"Test Case 1: nums=[2, 7, 11, 15], target=9 -> {two_sum([2, 7, 11, 15], 9)}")\nprint(f"Test Case 2: nums=[3, 2, 4], target=6 -> {two_sum([3, 2, 4], 6)}")\nprint(f"Test Case 3: nums=[3, 3], target=6 -> {two_sum([3, 3], 6)}")\n'},
  'id': '5adb7a22-6c2a-4210-98b9-9777246580b2',
  'type': 'tool_call'}]

In [67]:
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [68]:
from langchain_tavily import TavilySearch

In [69]:
from langchain_core.tools import tool

@tool(name_or_callable="tavily_search", description="Use Tavily to search the web for current or recent events.")
def tavily_search(query: str) -> str:
    search = TavilySearch(  
        max_results=5,
        topic="general",
        )
    response = search.invoke({"query": query})
    # Return a summary of top result(s) — you could customize this
    results = response.get("results", [])
    if not results:
        return "No results found."
    return results[0].get("content", "No content available.")

In [70]:
def generate_hint(question: str) -> str:
    """Generate a helpful hint for a DSA problem without solving it.
    input: question (str): The DSA problem to generate a hint for.
    output: str: A helpful hint for the DSA problem."""
    
    return llm.invoke(f"Give a helpful hint for this DSA problem without solving it: {question}")


In [71]:
def generate_test_cases(problem_description: str) -> str:
    """Use this to generate test cases for DSA problems.
    input: problem_description - the DSA problem statement
    output: 3 test cases without solving the problem"""
    
    return llm.invoke(f"Create 3 test cases for this DSA problem without solving it: {problem_description}")


In [72]:
def bug_hint_tool(code: str) -> str:
    """Use this to analyze code for logic issues and provide a subtle hint.
    input: code - the code to analyze
    output: str - a subtle hint about potential logic issues in the code.
    """

    return llm.invoke(f"Analyze this code for logic issues and give a subtle hint: {code}")


In [73]:
llm_with_tools = llm.bind_tools([repl_tool, tavily_search,generate_hint, generate_test_cases, bug_hint_tool])

In [74]:
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langgraph.graph import StateGraph, END

In [75]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


In [76]:
from typing import Callable

# You must register your tool functions in a dictionary
tool_map: dict[str, Callable] = {
    "python_repl": python_repl.run,
    "tavily_search": tavily_search,
    "generate_hint": generate_hint,
    "generate_test_cases": generate_test_cases,
    "bug_hint_tool": bug_hint_tool,
}

In [77]:
def execute_tools(state: State) -> State:
    last_message = state["messages"][-1]
    tool_outputs = []
    for tool_call in last_message.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        # Get function from registered tool map
        tool_func = tool_map.get(tool_name)
        if tool_func is None:
            output = f"[ERROR] Unknown tool: {tool_name}"
        else:
            try:
                # Handle different argument structures for different tools
                if tool_name == "python_repl":
                    # For python_repl, extract the string from __arg1 key
                    if "__arg1" in tool_args:
                        output = tool_func(tool_args["__arg1"])
                    else:
                        # If no __arg1, try to get the first value or convert dict to string
                        output = tool_func(str(tool_args))
                elif tool_name == "tavily_search":
                    # For tavily_search, it expects a query string
                    if "query" in tool_args:
                        output = tool_func(tool_args["query"])
                    else:
                        output = tool_func(str(tool_args))
                else:
                    # For other tools that expect string arguments
                    if len(tool_args) == 1:
                        # If only one argument, use its value
                        output = tool_func(list(tool_args.values())[0])
                    else:
                        # For multiple arguments, pass the whole dict
                        output = tool_func(tool_args)
            except Exception as e:
                output = f"[ERROR while running {tool_name}]: {e}"
        tool_outputs.append(
            ToolMessage(
                tool_call_id=tool_call["id"],
                content=str(output)  # Ensure output is always a string
            )
        )
    return {"messages": state["messages"] + tool_outputs}

In [78]:
def should_call_tool(state: State) -> str:
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "call_tool"
    return END

In [79]:
def call_llm(state: State) -> State:
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

In [80]:
graph = StateGraph(State)
graph.add_node("call_llm", call_llm)
graph.add_node("call_tool", execute_tools)
graph.set_entry_point("call_llm")
graph.add_conditional_edges("call_llm", should_call_tool)
graph.add_edge("call_tool", "call_llm")
app = graph.compile()

In [81]:
inputs = {
    "messages": [HumanMessage(content="What is two sum problem? generate test cases and run the test cases on code you generate,show output with proper debugging.")]
}
final_state = app.invoke(inputs)

In [82]:
from pprint import pprint

for msg in final_state["messages"]:
    print("="*30, msg.type.upper(), "="*30)
    if hasattr(msg, "pretty_print"):
        msg.pretty_print()
    else:
        pprint(msg)


============================== HUMAN ==============================
================================ Human Message =================================

What is two sum problem? generate test cases and run the test cases on code you generate,show output with proper debugging.
============================== AI ==============================
================================== Ai Message ==================================

The Two Sum problem is a classic problem in computer science. Given an array of integers `nums` and an integer `target`, the goal is to find the indices of two numbers in the array such that they add up to the `target`. You can assume that each input will have exactly one solution, and you may not use the same element twice.

Let's generate some test cases for the Two Sum problem.
Tool Calls:
  generate_test_cases (f6f2d536-aeda-40f3-ad42-8e7f8d08b0d3)
 Call ID: f6f2d536-aeda-40f3-ad42-8e7f8d08b0d3
  Args:
    problem_description: Given an array of integers `nums` and an i